In [85]:
import numpy as np
import pandas as pd
import math

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score


#linear models
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, RidgeCV, LassoCV, ElasticNetCV
from sklearn.svm import SVR

#ensembles
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost.sklearn import XGBRegressor

#knn
from sklearn.neighbors import KNeighborsRegressor

#neural networks
from sklearn.neural_network import MLPRegressor

from sktime.utils.data_processing import from_2d_array_to_nested
from sktime.transformations.panel.rocket import Rocket, MiniRocket


In [86]:
#load data
#data already in csv format
df = pd.read_csv("train.csv")
print(df.columns[:3])

Index(['kappa_casein', 'Casein_micelle_size', 'Native_pH'], dtype='object')


In [87]:
df.iloc[:,:3].corr()

,kappa_casein,Casein_micelle_size,Native_pH
kappa_casein,1.000000,0.112205,-0.080324
Casein_micelle_size,0.112205,1.000000,0.014424
Native_pH,-0.080324,0.014424,1.000000


# Visualization



In [191]:
import plotly.graph_objects as go

x_range = [x for x in range(test_df.shape[1])]
# Create traces
fig = go.Figure()
for index, row in test_df.iloc[:100,:].iterrows():
    fig.add_trace(go.Scatter(x=x_range, y=row[3:],
                    mode='lines',
                    name='lines'))
    
# fig.add_trace(go.Scatter(x=[600, 600], y=[-1, 4],
#     fill=None,
#     mode='lines',
#     line_color='indigo',
#     ))
# fig.add_trace(go.Scatter(
#     x=[830,830],
#     y=[-1, 4],
#     fill='tonexty', # fill area between trace0 and trace1
#     mode='lines', line_color='indigo'))

# fig.update_yaxes(
#     range=(-0.5, 4),
#     constrain='domain'
# )

fig.show()

In [89]:
# extract X and y
target_feature = "Casein_micelle_size"
tmp_df = df.dropna(subset=[target_feature])
print("Rows remained: " + str(tmp_df.shape[0]))

# filter with standard deviation
outlier_filter = tmp_df[target_feature] <= tmp_df[target_feature].mean() + 3 * tmp_df[target_feature].std()
# lowerbound filter - taken out to be comparable with Georgiana's exps
#outlier_filter = outlier_filter & (tmp_df[target_feature] >= (tmp_df[target_feature].mean() - 3 * tmp_df[target_feature].std()))
tmp_df = tmp_df[outlier_filter]
print("Rows remained: " + str(tmp_df.shape[0]))

#filter 
# outlier_filter = tmp_df[target_feature] <= 250.3
# tmp_df = tmp_df[outlier_filter]
# print("Rows remained: " + str(tmp_df.shape[0]))

#shuffle
tmp_df = tmp_df.sample(frac=1,random_state=0)
ts_length = 1060

X = tmp_df.iloc[:,3:(3+ts_length)]
Y = tmp_df[target_feature]

test_df = pd.read_csv("test.csv")
test_df = test_df.iloc[:,:ts_length]

Rows remained: 538
Rows remained: 526


# MiniROCKET

In [52]:
from sklearn.base import BaseEstimator,RegressorMixin

class MiniRocketRegr(BaseEstimator, RegressorMixin):
    def fit(self, X, y):
        X_sktime = from_2d_array_to_nested(X)    
        self.rocket = MiniRocket() 
        self.rocket.fit(X_sktime)
        X_train_transform = self.rocket.transform(X_sktime)
        self.regr = RidgeCV(alphas=np.logspace(-3, 3, 10),normalize=True)
        self.regr.fit(X_train_transform,y)            
        
        
        return self
    def predict(self, X):        
                             
        X_sktime = from_2d_array_to_nested(X)            

        X_transform = self.rocket.transform(X_sktime)           
        y_pred = self.regr.predict(X_transform)           
        
        
        return y_pred 

## Single split experiments

In [195]:
emr = MiniRocketRegr()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=0)
emr.fit(X_train,y_train)
y_pred = emr.predict(X_test)
print(math.sqrt(mean_squared_error(y_test, y_pred)))
print(r2_score(y_test, y_pred))

0.0788255031849735
0.415516211642606


In [196]:
import plotly.express as px
fig = px.scatter(x=y_test, y=y_pred)
fig.show()

# Cross validation exp

In [ ]:
scores = cross_validate(MiniRocketRegr(), X, Y, scoring=error_measures, cv=4)
rmse = np.mean(np.sqrt(-scores['test_neg_mean_squared_error']))
mae = np.mean(-scores['test_neg_mean_absolute_error'])
r2 = np.mean(scores['test_r2'])
print("RMSE: " + str(rmse))
print("R2: " + str(r2))

In [ ]:
#X_sktime = from_2d_array_to_nested(X)#.sample(frac=1).reset_index(drop=True)


regr = MiniRocketRegr().fit(X,Y)
y_pred = regr.predict(test_df)

In [ ]:
np.savetxt(target_feature + ".csv", y_pred, delimiter=",",fmt='%.3f')

# ENSEMBLE MiniROCKET


In [ ]:
from sklearn.base import BaseEstimator,RegressorMixin

class EnsembleMiniRocket(BaseEstimator, RegressorMixin):
    def fit(self, X, y):
        self.ensize = 100
        self.starts = np.random.randint(800, size=self.ensize)
        self.ends = self.starts + 100 + np.random.randint(150, size=self.ensize)   
        
        self.cpn = []

        for s,end in zip(self.starts,self.ends):                        
            X_sktime = from_2d_array_to_nested(X.iloc[:,s:end])            
            rocket = MiniRocket() 
            rocket.fit(X_sktime)
            X_train_transform = rocket.transform(X_sktime)
            regr = RidgeCV(alphas=np.logspace(-3, 3, 10),normalize=True)
            regr.fit(X_train_transform,y)            
            self.cpn.append([rocket,regr])
        
        return self
    def predict(self, X):        
        y_pred = np.zeros(X.shape[0])
        for s,end,c in zip(self.starts,self.ends, self.cpn):                        
            X_sktime = from_2d_array_to_nested(X.iloc[:,s:end])            
            rocket = c[0]
            regr = c[1]
            X_transform = rocket.transform(X_sktime)           
            y_pred = y_pred + regr.predict(X_transform)           
        
        
        return y_pred / self.ensize

In [ ]:
emr = EnsembleMiniRocket()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=0)
emr.fit(X_train,y_train)
y_pred = emr.predict(X_test)
print(math.sqrt(mean_squared_error(y_test, y_pred)))
print(r2_score(y_test, y_pred))

In [ ]:
scores = cross_validate(EnsembleMiniRocket(), X, Y, scoring=error_measures, cv=4)
rmse = np.mean(np.sqrt(-scores['test_neg_mean_squared_error']))
mae = np.mean(-scores['test_neg_mean_absolute_error'])
r2 = np.mean(scores['test_r2'])
print("RMSE: " + str(rmse))
print("R2: " + str(r2))

In [ ]:

regr = EnsembleMiniRocket().fit(X,Y)
y_pred = regr.predict(test_df)

In [ ]:
np.savetxt("prediction/EnsembleMiniRocket/" + target_feature + ".csv", y_pred, delimiter=",",fmt='%.6f')

## Optimized EnsembleMiniRocket

### with training fitness

In [ ]:
from sklearn.base import BaseEstimator,RegressorMixin

class EnsembleFittestMiniRocket(BaseEstimator, RegressorMixin):
    def fit(self, X, y):
        self.ensize = 100
        self.n_select = 20
        self.starts = np.random.randint(800, size=self.ensize)
        self.ends = self.starts + 100 + np.random.randint(150, size=self.ensize)   
        
        self.cpn = []
        self.scores = []

        for s,end in zip(self.starts,self.ends):                        
            X_sktime = from_2d_array_to_nested(X.iloc[:,s:end])            
            rocket = MiniRocket() 
            rocket.fit(X_sktime)
            X_train_transform = rocket.transform(X_sktime)
            regr = RidgeCV(alphas=np.logspace(-3, 3, 10),normalize=True)
            regr.fit(X_train_transform,y)            
            self.cpn.append([rocket,regr])
            score = math.sqrt(mean_squared_error(y, regr.predict(X_train_transform)))
#             score = -r2_score(y_test, y_pred)
            self.scores.append(score)
            
        selected_index = np.argsort(self.scores)[:self.n_select]
        
        self.starts = [self.starts[i] for i in selected_index]
        self.ends = [self.ends[i] for i in selected_index]
        self.cpn = [self.cpn[i] for i in selected_index]
        self.scores = [self.scores[i] for i in selected_index]
        #print(self.scores)
        
        return self
    def predict(self, X):        
        y_pred = np.zeros(X.shape[0])
        for s,end,c in zip(self.starts,self.ends, self.cpn):                        
            X_sktime = from_2d_array_to_nested(X.iloc[:,s:end])            
            rocket = c[0]
            regr = c[1]
            X_transform = rocket.transform(X_sktime)           
            y_pred = y_pred + regr.predict(X_transform)           
        
        
        return y_pred / len(self.cpn)

### with cross validation

In [204]:
from sklearn.base import BaseEstimator,RegressorMixin

class EnsembleMiniRocketCV(BaseEstimator, RegressorMixin):
    
    def cross_validate_score(self, X, y):
        scores = cross_validate(RidgeCV(alphas=np.logspace(-3, 3, 10),normalize=True), X, y, scoring=['neg_mean_squared_error'], cv=4)        
        return np.mean(np.sqrt(-scores['test_neg_mean_squared_error']))
    
    def fit(self, X, y):
        self.ensize = 100
        self.n_select = 50
        self.starts = np.random.randint(800, size=self.ensize)
        self.ends = self.starts + 100 + np.random.randint(150, size=self.ensize)   
        
        self.cpn = []
        self.scores = []

        for s,end in zip(self.starts,self.ends):                        
            X_sktime = from_2d_array_to_nested(X[:,s:end])            
            rocket = MiniRocket() 
            rocket.fit(X_sktime)
            X_train_transform = rocket.transform(X_sktime)
            regr = RidgeCV(alphas=np.logspace(-3, 3, 10),normalize=True)
            regr.fit(X_train_transform,y)            
            self.cpn.append([rocket,regr])
            score = self.cross_validate_score(X_train_transform,y)#           
            self.scores.append(score)
            
        selected_index = np.argsort(self.scores)[:self.n_select]
        
        self.starts = [self.starts[i] for i in selected_index]
        self.ends = [self.ends[i] for i in selected_index]
        self.cpn = [self.cpn[i] for i in selected_index]
        self.scores = [self.scores[i] for i in selected_index]
        #print(self.scores)
        
        return self
    def predict(self, X):        
        y_pred = np.zeros(X.shape[0])
        for s,end,c in zip(self.starts,self.ends, self.cpn):                        
            X_sktime = from_2d_array_to_nested(X[:,s:end])            
            rocket = c[0]
            regr = c[1]
            X_transform = rocket.transform(X_sktime)           
            y_pred = y_pred + regr.predict(X_transform)           
        
        
        return y_pred / len(self.cpn)

### weights with cross validation 

In [ ]:
from sklearn.base import BaseEstimator,RegressorMixin

class WeightedEnsembleMiniRocketCV(BaseEstimator, RegressorMixin):
    
    def cross_validate_score(self, X, y):
        scores = cross_validate(RidgeCV(alphas=np.logspace(-3, 3, 10),normalize=True), X, y, scoring=['neg_mean_squared_error'], cv=4)        
        return np.mean(np.sqrt(-scores['test_neg_mean_squared_error']))
    
    def fit(self, X, y):
        self.ensize = 100        
        self.starts = np.random.randint(800, size=self.ensize)
        self.ends = self.starts + 100 + np.random.randint(150, size=self.ensize)   
        
        self.cpn = []
        self.scores = []

        for s,end in zip(self.starts,self.ends):                        
            X_sktime = from_2d_array_to_nested(X.iloc[:,s:end])            
            rocket = MiniRocket() 
            rocket.fit(X_sktime)
            X_train_transform = rocket.transform(X_sktime)
            regr = RidgeCV(alphas=np.logspace(-3, 3, 10),normalize=True)
            regr.fit(X_train_transform,y)            
            self.cpn.append([rocket,regr])
            score = self.cross_validate_score(X_train_transform,y)#           
            self.scores.append(score)
            
        selected_index = np.argsort(self.scores)
        #weights = [0.5 + i/100 for i in range(101)]
        
        self.starts = [self.starts[i] for i in selected_index]
        self.ends = [self.ends[i] for i in selected_index]
        self.cpn = [self.cpn[i] for i in selected_index]
        self.scores = [self.scores[i] for i in selected_index]
        #print(self.scores)
        
        return self
    def predict(self, X):        
        y_pred = np.zeros(X.shape[0])
        weight = 200
        total_weight = 0
        for s,end,c in zip(self.starts,self.ends, self.cpn):                        
            X_sktime = from_2d_array_to_nested(X.iloc[:,s:end])            
            rocket = c[0]
            regr = c[1]
            X_transform = rocket.transform(X_sktime)           
            y_pred = y_pred + regr.predict(X_transform)*weight
            total_weight += weight
            weight -= 2
        
        
        return y_pred / total_weight

# Ensemble from sklearn

## Individual Interval MiniRocket

In [76]:
from sklearn.base import BaseEstimator,RegressorMixin

class IntvMiniRocketRegr(BaseEstimator, RegressorMixin):
    def fit(self, X, y):
        self.start = np.random.randint(800)
        self.end = self.start + 100 + np.random.randint(150)   
        X_sktime = from_2d_array_to_nested(X[:,self.start:self.end])    
        self.rocket = MiniRocket() 
        self.rocket.fit(X_sktime)
        X_train_transform = self.rocket.transform(X_sktime)
        self.regr = RidgeCV(alphas=np.logspace(-3, 3, 10),normalize=True)
        self.regr.fit(X_train_transform,y)            
        
        return self
    def predict(self, X):                                   
        
        X_sktime = from_2d_array_to_nested(X[:,self.start:self.end])    
        X_transform = self.rocket.transform(X_sktime)           
        y_pred = self.regr.predict(X_transform)           
        
        
        return y_pred 
    


## Voting Regressor

In [97]:
from sklearn.ensemble import VotingRegressor

emr = VotingRegressor([('mrr' + str(i),IntvMiniRocketRegr()) for i in range(100)] )

## StackRegressor 

In [70]:
from sklearn.ensemble import StackingRegressor


emr = StackingRegressor(
        estimators=[('mrr' + str(i),IntvMiniRocketRegr()) for i in range(10)],
        final_estimator=RandomForestRegressor(n_estimators=10,
                                              random_state=42)
)   

## BaggingRegressor

In [77]:
from sklearn.ensemble import BaggingRegressor
emr = BaggingRegressor(base_estimator=IntvMiniRocketRegr(),
            n_estimators=100, random_state=0)

## AdaBoost Regressor

In [81]:
from sklearn.ensemble import AdaBoostRegressor
emr = AdaBoostRegressor(base_estimator=IntvMiniRocketRegr(),random_state=0, n_estimators=100)

In [82]:
#emr = MiniRocketRegr()
X_train, X_test, y_train, y_test = train_test_split(X.values, Y, test_size=0.25, random_state=0)
emr.fit(X_train,y_train)
y_pred = emr.predict(X_test)
print(math.sqrt(mean_squared_error(y_test, y_pred)))
print(r2_score(y_test, y_pred))

0.07377795156346706
0.4879738376973125


In [79]:
error_measures = ['neg_mean_squared_error','r2']
scores = cross_validate(emr, X, Y, scoring=error_measures, cv=4)
rmse = np.mean(np.sqrt(-scores['test_neg_mean_squared_error']))
#mae = np.mean(-scores['test_neg_mean_absolute_error'])
r2 = np.mean(scores['test_r2'])
print("RMSE: " + str(rmse))
print("R2: " + str(r2))

RMSE: 0.0748177127004154
R2: 0.5033710666963042


## EXPERIMENT

In [106]:
def extract_trait_data(target_feature):
    # extract X and y    
    tmp_df = df.dropna(subset=[target_feature])
    print("Rows remained: " + str(tmp_df.shape[0]))

    # filter with standard deviation
    outlier_filter = tmp_df[target_feature] <= tmp_df[target_feature].mean() + 3 * tmp_df[target_feature].std()
    # lowerbound filter - taken out to be comparable with Georgiana's exps
    #outlier_filter = outlier_filter & (tmp_df[target_feature] >= (tmp_df[target_feature].mean() - 3 * tmp_df[target_feature].std()))
    tmp_df = tmp_df[outlier_filter]
    print("Rows remained: " + str(tmp_df.shape[0]))

    #filter 
    # outlier_filter = tmp_df[target_feature] <= 250.3
    # tmp_df = tmp_df[outlier_filter]
    # print("Rows remained: " + str(tmp_df.shape[0]))

    #shuffle
    tmp_df = tmp_df.sample(frac=1,random_state=0)    

    X = tmp_df.iloc[:,3:]
    Y = tmp_df[target_feature]

    return X, Y

In [ ]:
for tf in ['kappa_casein', 'Casein_micelle_size', 'Native_pH']:
    print("=======================")
    print("Trait: " + str(tf)) 
    
    X,Y = extract_trait_data(tf)
    #rgr = StackingRegressor(
    #    estimators=[('mrr' + str(i),IntvMiniRocketRegr()) for i in range(100)],
    #    final_estimator=RidgeCV(alphas=np.logspace(-3, 3, 10),normalize=True))
    #rgr1 = VotingRegressor([('mrr' + str(i),IntvMiniRocketRegr()) for i in range(100)] )
    #rgr2 = MrSQMRegressor()
    #rgr3 = RidgeCV(alphas=np.logspace(-3, 3, 10),normalize=True)
    
    #er = VotingRegressor([('rk',rgr1),('sqm',rgr2),('tab',rgr3)])
    rgr = EnsembleMiniRocketCV()
    scores = cross_validate(rgr, X.values, Y, scoring=error_measures, cv=4)
    rmse = np.mean(np.sqrt(-scores['test_neg_mean_squared_error']))
    #mae = np.mean(-scores['test_neg_mean_absolute_error'])
    r2 = np.mean(scores['test_r2'])
    print("RMSE: " + str(rmse))
    print("R2: " + str(r2))

Trait: kappa_casein
Rows remained: 399
Rows remained: 396
RMSE: 1.1703181079544098
R2: 0.38856349184320316
Trait: Casein_micelle_size
Rows remained: 538
Rows remained: 526
RMSE: 56.87309292493206
R2: 0.004004038273085181
Trait: Native_pH
Rows remained: 548
Rows remained: 542
